In [2]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark= SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv009033/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [3]:
spark

# Question 3 : Use the dataset given in HDFS(path: /public/trendytech/retail_db/customers)

In [4]:
customers_df = spark.read.format('csv')\
                    .option('header', 'true')\
                    .option('inferschema','true')\
                    .option("mode", "DROPMALFORMED")\
                    .load("/user/itv009033/customers_folder/final_customers.csv")

In [5]:
spark.sql("show databases")

namespace
00000assign7_itv0...
0001_av_ivy_tesco
001_retail
003402_hive1
005198_ivy_tesco
005212_ivy_tesco
005222_ivy_practice
005260_ivy_database
005876_week5_assi...
005933_retail


In [6]:
customers_df.show(truncate = False)

+-------+-----------+----------+----------+-------------+---------------------------+-------------+----------+------------+
|cust_id|cust_fname |cust_lname|cust_email|cust_password|cust_street                |cust_city    |cust_state|cust_zipcode|
+-------+-----------+----------+----------+-------------+---------------------------+-------------+----------+------------+
|1      |Richard    |Hernandez |XXXXXXXXX |XXXXXXXXX    |6303 Heather Plaza         |Brownsville  |TX        |78521       |
|2      |Mary       |Barrett   |XXXXXXXXX |XXXXXXXXX    |9526 Noble Embers Ridge    |Littleton    |CO        |80126       |
|3      |Ann        |Smith     |XXXXXXXXX |XXXXXXXXX    |3422 Blue Pioneer Bend     |Caguas       |PR        |725         |
|4      |Mary       |Jones     |XXXXXXXXX |XXXXXXXXX    |8324 Little Common         |San Marcos   |CA        |92069       |
|5      |Robert     |Hudson    |XXXXXXXXX |XXXXXXXXX    |10 Crystal River Mall      |Caguas       |PR        |725         |
|6      

In [8]:
spark.sql("use itv009033_week5_assignment")

""


In [9]:
customers_df.printSchema()

root
 |-- cust_id: integer (nullable = true)
 |-- cust_fname: string (nullable = true)
 |-- cust_lname: string (nullable = true)
 |-- cust_email: string (nullable = true)
 |-- cust_password: string (nullable = true)
 |-- cust_street: string (nullable = true)
 |-- cust_city: string (nullable = true)
 |-- cust_state: string (nullable = true)
 |-- cust_zipcode: integer (nullable = true)



In [11]:
customers_df.createOrReplaceTempView("customer")

In [12]:
spark.sql("show tables").show(truncate = False)

+--------------------------+---------+-----------+
|database                  |tableName|isTemporary|
+--------------------------+---------+-----------+
|itv009033_week5_assignment|customers|false      |
|itv009033_week5_assignment|product  |false      |
|                          |customer |true       |
+--------------------------+---------+-----------+



In [13]:
spark.sql("drop table customers")

""


In [14]:
spark.sql("show tables").show(truncate = False)

+--------------------------+---------+-----------+
|database                  |tableName|isTemporary|
+--------------------------+---------+-----------+
|itv009033_week5_assignment|product  |false      |
|                          |customer |true       |
+--------------------------+---------+-----------+



In [15]:
spark.sql("create table if not exists itv009033_week5_assignment.customers(cust_id int, cust_fname string, cust_lname string, cust_email string, cust_password string, cust_street string, cust_city string, cust_state string, cust_zipcode int)")

""


In [16]:
spark.sql("show tables").show(truncate = False)

+--------------------------+---------+-----------+
|database                  |tableName|isTemporary|
+--------------------------+---------+-----------+
|itv009033_week5_assignment|customers|false      |
|itv009033_week5_assignment|product  |false      |
|                          |customer |true       |
+--------------------------+---------+-----------+



In [17]:
spark.sql("insert into itv009033_week5_assignment.customers select * from customer").show()

++
||
++
++



In [18]:
spark.sql("select * from customers").show(truncate = False)

+-------+-----------+----------+----------+-------------+---------------------------+-------------+----------+------------+
|cust_id|cust_fname |cust_lname|cust_email|cust_password|cust_street                |cust_city    |cust_state|cust_zipcode|
+-------+-----------+----------+----------+-------------+---------------------------+-------------+----------+------------+
|1      |Richard    |Hernandez |XXXXXXXXX |XXXXXXXXX    |6303 Heather Plaza         |Brownsville  |TX        |78521       |
|2      |Mary       |Barrett   |XXXXXXXXX |XXXXXXXXX    |9526 Noble Embers Ridge    |Littleton    |CO        |80126       |
|3      |Ann        |Smith     |XXXXXXXXX |XXXXXXXXX    |3422 Blue Pioneer Bend     |Caguas       |PR        |725         |
|4      |Mary       |Jones     |XXXXXXXXX |XXXXXXXXX    |8324 Little Common         |San Marcos   |CA        |92069       |
|5      |Robert     |Hudson    |XXXXXXXXX |XXXXXXXXX    |10 Crystal River Mall      |Caguas       |PR        |725         |
|6      

# 3.1. Find the total number of customers in each state.

In [19]:
customers_df.select("cust_state").groupBy("cust_state").count().sort("count", ascending = False).show()

+----------+-----+
|cust_state|count|
+----------+-----+
|        PR| 4771|
|        CA| 2012|
|        NY|  775|
|        TX|  635|
|        IL|  523|
|        FL|  374|
|        OH|  276|
|        PA|  261|
|        MI|  254|
|        NJ|  219|
|        AZ|  213|
|        GA|  169|
|        MD|  164|
|        NC|  150|
|        VA|  136|
|        CO|  122|
|        OR|  119|
|        MA|  113|
|        TN|  104|
|        NV|  103|
+----------+-----+
only showing top 20 rows



In [20]:
spark.sql("select cust_state, count(*) as count_of_states from customers group by 1 order by count(*) desc").show()

+----------+---------------+
|cust_state|count_of_states|
+----------+---------------+
|        PR|           4771|
|        CA|           2012|
|        NY|            775|
|        TX|            635|
|        IL|            523|
|        FL|            374|
|        OH|            276|
|        PA|            261|
|        MI|            254|
|        NJ|            219|
|        AZ|            213|
|        GA|            169|
|        MD|            164|
|        NC|            150|
|        VA|            136|
|        CO|            122|
|        OR|            119|
|        MA|            113|
|        TN|            104|
|        NV|            103|
+----------+---------------+
only showing top 20 rows



# 3.2. Find the top 5 most common last names among the customers

In [21]:
customers_df.select("cust_lname").groupBy("cust_lname").count().sort("count",ascending = False).limit(5).show(truncate = False)

+----------+-----+
|cust_lname|count|
+----------+-----+
|Smith     |4626 |
|Johnson   |76   |
|Williams  |69   |
|Jones     |65   |
|Brown     |62   |
+----------+-----+



In [22]:
spark.sql("select cust_lname, count(cust_lname) as last_name_counter from customers group by 1 order by last_name_counter desc limit 5").show()

+----------+-----------------+
|cust_lname|last_name_counter|
+----------+-----------------+
|     Smith|             4626|
|   Johnson|               76|
|  Williams|               69|
|     Jones|               65|
|     Brown|               62|
+----------+-----------------+



# 3.3. Check whether there are any customers whose zip codes are not valid (i.e., not equal to 5 digits).


In [23]:
from pyspark.sql.functions import length

In [24]:
customers_df.select("*").filter(length("cust_zipcode")!=5).show()

+-------+----------+----------+----------+-------------+--------------------+-------------+----------+------------+
|cust_id|cust_fname|cust_lname|cust_email|cust_password|         cust_street|    cust_city|cust_state|cust_zipcode|
+-------+----------+----------+----------+-------------+--------------------+-------------+----------+------------+
|      3|       Ann|     Smith| XXXXXXXXX|    XXXXXXXXX|3422 Blue Pioneer...|       Caguas|        PR|         725|
|      5|    Robert|    Hudson| XXXXXXXXX|    XXXXXXXXX|10 Crystal River ...|       Caguas|        PR|         725|
|      6|      Mary|     Smith| XXXXXXXXX|    XXXXXXXXX|3151 Sleepy Quail...|      Passaic|        NJ|        7055|
|      7|   Melissa|    Wilcox| XXXXXXXXX|    XXXXXXXXX|9453 High Concession|       Caguas|        PR|         725|
|      8|     Megan|     Smith| XXXXXXXXX|    XXXXXXXXX|3047 Foggy Forest...|     Lawrence|        MA|        1841|
|      9|      Mary|     Perez| XXXXXXXXX|    XXXXXXXXX| 3616 Quaking St

In [25]:
spark.sql("select * from customers where LENGTH(cust_zipcode)<>5").show()

+-------+----------+----------+----------+-------------+--------------------+-------------+----------+------------+
|cust_id|cust_fname|cust_lname|cust_email|cust_password|         cust_street|    cust_city|cust_state|cust_zipcode|
+-------+----------+----------+----------+-------------+--------------------+-------------+----------+------------+
|      3|       Ann|     Smith| XXXXXXXXX|    XXXXXXXXX|3422 Blue Pioneer...|       Caguas|        PR|         725|
|      5|    Robert|    Hudson| XXXXXXXXX|    XXXXXXXXX|10 Crystal River ...|       Caguas|        PR|         725|
|      6|      Mary|     Smith| XXXXXXXXX|    XXXXXXXXX|3151 Sleepy Quail...|      Passaic|        NJ|        7055|
|      7|   Melissa|    Wilcox| XXXXXXXXX|    XXXXXXXXX|9453 High Concession|       Caguas|        PR|         725|
|      8|     Megan|     Smith| XXXXXXXXX|    XXXXXXXXX|3047 Foggy Forest...|     Lawrence|        MA|        1841|
|      9|      Mary|     Perez| XXXXXXXXX|    XXXXXXXXX| 3616 Quaking St

# 3.4. Count the number of customers who have valid zip codes.

In [26]:
customers_df.select("*").filter(length("cust_zipcode")==5).show()

+-------+-----------+----------+----------+-------------+--------------------+-----------+----------+------------+
|cust_id| cust_fname|cust_lname|cust_email|cust_password|         cust_street|  cust_city|cust_state|cust_zipcode|
+-------+-----------+----------+----------+-------------+--------------------+-----------+----------+------------+
|      1|    Richard| Hernandez| XXXXXXXXX|    XXXXXXXXX|  6303 Heather Plaza|Brownsville|        TX|       78521|
|      2|       Mary|   Barrett| XXXXXXXXX|    XXXXXXXXX|9526 Noble Embers...|  Littleton|        CO|       80126|
|      4|       Mary|     Jones| XXXXXXXXX|    XXXXXXXXX|  8324 Little Common| San Marcos|        CA|       92069|
|     10|    Melissa|     Smith| XXXXXXXXX|    XXXXXXXXX|8598 Harvest Beac...|   Stafford|        VA|       22554|
|     12|Christopher|     Smith| XXXXXXXXX|    XXXXXXXXX|5594 Jagged Ember...|San Antonio|        TX|       78227|
|     14|  Katherine|     Smith| XXXXXXXXX|    XXXXXXXXX|5666 Hazy Pony Sq...|Pi

In [27]:
customers_df.select("*").filter(length("cust_zipcode")==5).count()

7244

In [28]:
spark.sql("select * from customers where LENGTH(cust_zipcode)=5").show()

+-------+-----------+----------+----------+-------------+--------------------+-----------+----------+------------+
|cust_id| cust_fname|cust_lname|cust_email|cust_password|         cust_street|  cust_city|cust_state|cust_zipcode|
+-------+-----------+----------+----------+-------------+--------------------+-----------+----------+------------+
|      1|    Richard| Hernandez| XXXXXXXXX|    XXXXXXXXX|  6303 Heather Plaza|Brownsville|        TX|       78521|
|      2|       Mary|   Barrett| XXXXXXXXX|    XXXXXXXXX|9526 Noble Embers...|  Littleton|        CO|       80126|
|      4|       Mary|     Jones| XXXXXXXXX|    XXXXXXXXX|  8324 Little Common| San Marcos|        CA|       92069|
|     10|    Melissa|     Smith| XXXXXXXXX|    XXXXXXXXX|8598 Harvest Beac...|   Stafford|        VA|       22554|
|     12|Christopher|     Smith| XXXXXXXXX|    XXXXXXXXX|5594 Jagged Ember...|San Antonio|        TX|       78227|
|     14|  Katherine|     Smith| XXXXXXXXX|    XXXXXXXXX|5666 Hazy Pony Sq...|Pi

In [29]:
spark.sql("select count(*) as valid_zips from customers where LENGTH(cust_zipcode)=5").show()

+----------+
|valid_zips|
+----------+
|      7244|
+----------+



# 3.5. Find the number of customers from each city in the state of California(CA).

In [30]:
customers_df.select("cust_city").filter("cust_state == 'CA'").groupBy("cust_city").count().sort("count", ascending =  False).show()

+--------------+-----+
|     cust_city|count|
+--------------+-----+
|   Los Angeles|  224|
|     San Diego|  104|
|      San Jose|   71|
|   Bakersfield|   41|
|     Santa Ana|   36|
|    Long Beach|   36|
|        Fresno|   29|
|     Escondido|   29|
|       Ontario|   29|
| San Francisco|   28|
|     Riverside|   27|
| Mission Viejo|   26|
|     Oceanside|   24|
|    Sacramento|   23|
|       Modesto|   23|
|       Fremont|   22|
|       Hayward|   21|
| Moreno Valley|   21|
|   Simi Valley|   20|
|San Bernardino|   20|
+--------------+-----+
only showing top 20 rows



In [31]:
spark.sql("select cust_city, count(*) as count_of_cities from customers where cust_state = 'CA' group by 1 order by count_of_cities desc").show()

+-------------+---------------+
|    cust_city|count_of_cities|
+-------------+---------------+
|  Los Angeles|            224|
|    San Diego|            104|
|     San Jose|             71|
|  Bakersfield|             41|
|    Santa Ana|             36|
|   Long Beach|             36|
|       Fresno|             29|
|    Escondido|             29|
|      Ontario|             29|
|San Francisco|             28|
|    Riverside|             27|
|Mission Viejo|             26|
|    Oceanside|             24|
|   Sacramento|             23|
|      Modesto|             23|
|      Fremont|             22|
|Moreno Valley|             21|
|      Hayward|             21|
| Garden Grove|             20|
|  Simi Valley|             20|
+-------------+---------------+
only showing top 20 rows

